## Import the Packages

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import xlrd
import datetime
from sklearn.neural_network import MLPRegressor  # 多层线性回归
from sklearn.preprocessing import StandardScaler
from pandas._libs import tslibs 
import random

## Import the DataSet

In [7]:
#Import the data_set
data_month = []
for i in range(19):
    data_month.append(pd.read_csv('DataProcessed/Data_month_{}.csv'.format(str(i))) );

## Add the feature month

In [14]:
month_seq = [5,6,7,8,9,10,12,1,2,3,4,5,6,7,8,9,10,11]
for j in range(18):
    month = [];
    for i in range(1, data_month[j].shape[0]):
        month.append(month_seq[j])
    Month = pd.DataFrame(month)
    data_month[j] = pd.merge(data_month[j], Month,left_index=True,right_index=True,how='outer')

## Concatenate the DataSet

In [19]:
raw_data = data_month[0];
for i in range(1,19):
    raw_data = pd.concat([raw_data, data_month[i]], ignore_index=True)
raw_data.columns = [u'Time_hour',u'AllEnergy',u'Month']

C:\Users\Automne\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
#The data concatenated all together;
print(raw_data)

       Unnamed: 0           0    0
0               0  303.500000  5.0
1               1  278.000000  5.0
2               2  278.000000  5.0
3               3  258.500000  5.0
4               4  228.500000  5.0
5               5  355.500000  5.0
6               6  395.500000  5.0
7               7  523.500000  5.0
8               8  462.500000  5.0
9               9  521.500000  5.0
10             10  660.500000  5.0
11             11  570.000000  5.0
12             12  324.000000  5.0
13             13  322.500000  5.0
14             14  572.000000  5.0
15             15  405.000000  5.0
16             16  690.000000  5.0
17             17  524.500000  5.0
18             18  499.500000  5.0
19             19  680.500000  5.0
20             20  710.500000  5.0
21             21  588.500000  5.0
22             22  499.000000  5.0
23             23  335.000000  5.0
24             24  259.250000  5.0
25             25  217.000000  5.0
26             26  240.000000  5.0
27             27  2

## Add the different features

In [20]:
#Add the feature hour in day range from 0 to 23
raw_data['Hour_in_Day'] = raw_data['Time_hour']%24

#Add the feature day in month range from 1 to 31;
day_in_month = [];
day_in_year = [];
for i in range(1, raw_data.shape[0]):
    day_in_month.append( 1+int((raw_data['Time_hour'][i])/24))
    day_in_year.append(1+int( i /24))

Day_in_month = pd.DataFrame(day_in_month)
Day_in_year = pd.DataFrame(day_in_year)
Energy_per_hour = pd.merge(raw_data, Day_in_month ,left_index=True,right_index=True,how='outer')
Energy_per_hour = pd.merge(Energy_per_hour, Day_in_year ,left_index=True,right_index=True,how='outer')
Energy_per_hour.columns = ['Time_hour', 'AllEnergy','Month', 'Hour_in_Day', 'Day_in_month', 'Day_in_year']

#Add the feature date in week;
Energy_per_hour['Day_in_Week'] = Energy_per_hour['Day_in_year']%7

#Add the feature whether the it's the weekend day or not;
Is_weekend = []
for i in range(1, Energy_per_hour.shape[0]):
    if(Energy_per_hour['Day_in_Week'][i] > 5 or Energy_per_hour['Day_in_Week'][i] < 1):
        Is_weekend.append(1)
    else:
        Is_weekend.append(0)
Is_Weekend = pd.DataFrame(Is_weekend)
Energy_per_hour = pd.merge(Energy_per_hour, Is_Weekend,left_index=True,right_index=True,how='outer')

Energy_per_hour.columns = ['Time_hour', 'AllEnergy', 'Month' ,'Hour_in_Day', 'Day_in_month', 'Day_in_year','Day_in_week', 'Is_weekend']

#Set the holiday date set
Holiday = [(1,1)]
for i in range(15,21):
    Holiday.append( (2,i) )
for i in range(5,7):
    Holiday.append( (4,i) )
for i in range(29,30):
    Holiday.append( (4,i) )
Holiday.append( (5,1) )
for i in range(16,18):
    Holiday.append( (6,i) )
for i in range(22,24):
    Holiday.append( (9,i) )
for i in range(1,7):
    Holiday.append( (10,i) )
    
Is_holiday = []

for i in range(1, Energy_per_hour.shape[0]):
    flag = 0;
    for j in range(0, len(Holiday)-1):
        if (  (Energy_per_hour['Month'][i], Energy_per_hour['Day_in_month'][i]) == Holiday[j]  ):
            flag = 1
    Is_holiday.append(flag)
Is_Holiday = pd.DataFrame(Is_holiday)
Energy_per_hour = pd.merge(Energy_per_hour, Is_Holiday,left_index=True,right_index=True,how='outer')
Energy_per_hour.columns = ['Time_hour', 'AllEnergy', 'Month' ,'Hour_in_Day', 'Day_in_month', 'Day_in_year','Day_in_week', 'Is_weekend', 'Is_Holiday']

#Add the energy consuming condition before as features
for h in range(1,50):
    Energy_last = []
    for i in range(0,h):
        Energy_last.append(0)
    for i in range(h, raw_data.shape[0]):
        Energy_last.append( raw_data['AllEnergy'][i - h])
    Energy_last = pd.DataFrame(Energy_last)
    Energy_per_hour = pd.merge(Energy_per_hour, Energy_last,left_index=True,right_index=True,how='outer')

## Import the Data to Training DateSet 

In [21]:
pd.DataFrame(Energy_per_hour).to_csv('TrainingData/Data_month_with_features_2.csv')